# Setup Amazon Comprehend Through AWS Console 

<img src="img/comprehend.png" width="80%" align="left">

## Note that Amazon Comprehend is currently only supported in a subset of regions: 

* US East (N. Virginia), US East (Ohio), US West (Oregon)
* Canada (Central)
* Europe (London), Europe (Ireland), Europe (Frankfurt)
* Asia Pacific (Mumbai), Asia Pacific (Seoul), Asia Pacific (Tokyo), Asia Pacific (Singapore), Asia Pacific (Sydney)

You can check https://aws.amazon.com/about-aws/global-infrastructure/regional-product-services/ for details and updates. 

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(
   retries = {
      'max_attempts': 10,
      'mode': 'adaptive'
   }
)

iam = boto3.client('iam', config=config)
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

### Check if you current regions supports Comprehend

In [2]:
if region in ['ap-south-1', 'eu-west-2', 'eu-west-1', 'ap-northeast-2', 'ap-northeast-1', 'ca-central-1', 'ap-southeast-1', 'ap-southeast-2', 'eu-central-1', 'us-east-1', 'us-east-2', 'us-west-2']:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )
    print(' SUCCESS: COMPREHEND IS SUPPORTED IN {}'.format(region))
    print(' Please proceed with this notebook.' )
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )
else:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )
    print(' !! COMPREHEND IS *NOT* SUPPORTED IN {}!! '.format(region))
    print(' This is OK. Skip this notebook and continue with the next notebook.' )
    print(' This notebook is not required for the rest of this workshop.' )
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 SUCCESS: COMPREHEND IS SUPPORTED IN us-west-2
 Please proceed with this notebook.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [3]:
comprehend = boto3.client('comprehend')

### Retrieve S3 location of training data

In [4]:
%store -r noheader_train_s3_uri

In [5]:
if not noheader_train_s3_uri:
    print('****************************************************************************************')
    print('**************** PLEASE RE-RUN THE PREVIOUS DATA PREPARATION NOTEBOOK ******************')
    print('**************** THIS NOTEBOOK WILL NOT RUN PROPERLY ***********************************')
    print('****************************************************************************************')

In [6]:
print(noheader_train_s3_uri)

s3://sagemaker-us-west-2-032934710550/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [7]:
!aws s3 ls $noheader_train_s3_uri

2020-08-22 15:45:47   13621088 amazon_reviews_us_Digital_Software_v1_00_noheader.csv


## See our prepared training data which we use as input for Comprehend

In [8]:
!aws s3 cp $noheader_train_s3_uri ./data/

download: s3://sagemaker-us-west-2-032934710550/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv to data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [9]:
import csv

df = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv', header=None)
df.head()

,0,1
0,2,I use to use Quicken 2006 on my Windows comput...
1,5,great product. easy installation and maintenan...
2,1,basically only download banking activity. Cant...
3,2,I'd normally rate this very high. I've used Q...
4,2,Each register update takes a long time. The c...


# Create Data Access Role for Comprehend

## Create Policy

In [10]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

## Create Role and Attach Policies

In [11]:
iam_comprehend_role_name = 'DSOAWS_Comprehend'

In [12]:
import json
import time

from botocore.exceptions import ClientError

try:
    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Comprehend Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)
        
time.sleep(30)

In [13]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        }
    ]
}

print(comprehend_s3_policy_doc)

{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-032934710550/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-032934710550'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-032934710550/*'], 'Effect': 'Allow'}]}


# Attach Policy to Role

In [14]:
import time

response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName='DSOAWS_ComprehendPolicyToS3',
    PolicyDocument=json.dumps(comprehend_s3_policy_doc)
)

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': '81c3e8e6-6cea-498e-963c-25560716dff0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '81c3e8e6-6cea-498e-963c-25560716dff0', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Sat, 22 Aug 2020 16:25:09 GMT'}, 'RetryAttempts': 0}}


# Train the Model

In [15]:
prefix = 'models'

s3_output_job = 's3://{}/{}/{}'.format(bucket, prefix, 'comprehend/output')
print(s3_output_job)

s3://sagemaker-us-west-2-032934710550/models/comprehend/output


In [16]:
iam_role_comprehend_arn = iam_role_comprehend['Role']['Arn']

In [17]:
import datetime
import time

timestamp = str(datetime.datetime.now().strftime("%s"))

comprehend_training_job_name = 'Amazon-Customer-Reviews-Classifier-{}'.format(timestamp) 

print(comprehend_training_job_name)

Amazon-Customer-Reviews-Classifier-1598113602


In [18]:
training_job = comprehend.create_document_classifier(
    DocumentClassifierName=comprehend_training_job_name,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={
        'S3Uri': noheader_train_s3_uri
    },
    OutputDataConfig={
        'S3Uri': s3_output_job
    },
    LanguageCode='en'
)

time.sleep(30)

In [20]:
comprehend_training_job_arn = training_job['DocumentClassifierArn']

print(comprehend_training_job_arn)

arn:aws:comprehend:us-west-2:032934710550:document-classifier/Amazon-Customer-Reviews-Classifier-1598113602


In [21]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/comprehend/v2/home?region={}#classifier-details/{}">Comprehend Training Job</a></b>'.format(region, comprehend_training_job_arn)))


In [22]:
import time


max_time = time.time() + 3 * 60 * 60 # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn = comprehend_training_job_arn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        print('')
        print('Status {}'.format(status))
        print('')
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break
        
    time.sleep(10)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom cla

Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: T

Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: T

# _Please Wait Until the ^^ Classifier ^^ is Trained Above._

# Show Results of the Classifier

In [23]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-west-2:032934710550:document-classifier/Amazon-Customer-Reviews-Classifier-1598113602', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2020, 8, 22, 16, 26, 58, 526000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 8, 22, 18, 13, 33, 522000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 8, 22, 16, 31, 25, 228000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 8, 22, 18, 12, 24, 211000, tzinfo=tzlocal()), 'InputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-032934710550/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-032934710550/models/comprehend/output/032934710550-CLR-ee14494740803a7efd64e18b55114023/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 27905, 'NumberOfTestDocuments': 3100, 'EvaluationMetrics': {'Accuracy': 0.5442, 'Precision': 0.5389, 'Rec

In [24]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-west-2:032934710550:document-classifier/Amazon-Customer-Reviews-Classifier-1598113602


In [25]:
import os
#Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = 's3://{}/'.format(bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-us-west-2-032934710550/models/comprehend/output/032934710550-CLR-ee14494740803a7efd64e18b55114023/output/output.tar.gz
models/comprehend/output/032934710550-CLR-ee14494740803a7efd64e18b55114023/output/output.tar.gz


# Download Model Artifacts including Training Metrics

In [26]:
s3 = boto3.resource('s3')

s3.Bucket(bucket).download_file(job_key, './output.tar.gz')

In [27]:
#Unpack the gzip file
!tar xvzf ./output.tar.gz

output/
output/confusion_matrix.json


In [28]:
import json

with open('./output/confusion_matrix.json') as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      414,
      124,
      59,
      13,
      10
    ],
    [
      160,
      252,
      153,
      40,
      15
    ],
    [
      53,
      146,
      276,
      118,
      27
    ],
    [
      20,
      35,
      118,
      288,
      159
    ],
    [
      12,
      11,
      22,
      118,
      457
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [29]:
!pip install tabulate

In [30]:
from IPython.display import HTML, display
import tabulate
table = [['', '1', '2', '3', '4', '5', '(Predicted)'],
         ['1', data['confusion_matrix'][0][0], data['confusion_matrix'][0][1], data['confusion_matrix'][0][2], data['confusion_matrix'][0][3], data['confusion_matrix'][0][4]],
         ['2', data['confusion_matrix'][1][0], data['confusion_matrix'][1][1], data['confusion_matrix'][1][2], data['confusion_matrix'][1][3], data['confusion_matrix'][1][4]],
         ['3', data['confusion_matrix'][2][0], data['confusion_matrix'][2][1], data['confusion_matrix'][2][2], data['confusion_matrix'][2][3], data['confusion_matrix'][2][4]],
         ['4', data['confusion_matrix'][3][0], data['confusion_matrix'][3][1], data['confusion_matrix'][3][2], data['confusion_matrix'][3][3], data['confusion_matrix'][3][4]],
         ['5', data['confusion_matrix'][4][0], data['confusion_matrix'][4][1], data['confusion_matrix'][4][2], data['confusion_matrix'][4][3], data['confusion_matrix'][4][4]],
         ['(Actual)']]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,1,2,3,4,5,(Predicted)
1,414,124,59,13,10,
2,160,252,153,40,15,
3,53,146,276,118,27,
4,20,35,118,288,159,
5,12,11,22,118,457,
(Actual),,,,,,


# Deploy Endpoint

In [31]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

comprehend_endpoint_name = 'comprehend-inference-ep-' + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name,
    ModelArn=model_arn,
    DesiredInferenceUnits=1
)

In [32]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-west-2:032934710550:document-classifier-endpoint/comprehend-inference-ep-22-18-13-36


# Pass Variables to the Next Notebook(s)

In [33]:
%store comprehend_endpoint_arn

Stored 'comprehend_endpoint_arn' (str)


# Predict with Endpoint

In [34]:
describe_response = comprehend.describe_endpoint(
    EndpointArn = comprehend_endpoint_arn
)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-west-2:032934710550:document-classifier-endpoint/comprehend-inference-ep-22-18-13-36', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-west-2:032934710550:document-classifier/Amazon-Customer-Reviews-Classifier-1598113602', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2020, 8, 22, 18, 13, 36, 504000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 8, 22, 18, 13, 36, 504000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': 'fb852dd4-8b6a-4f96-8106-145783ddc7b0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fb852dd4-8b6a-4f96-8106-145783ddc7b0', 'content-type': 'application/x-amz-json-1.1', 'content-length': '408', 'date': 'Sat, 22 Aug 2020 18:13:36 GMT'}, 'RetryAttempts': 0}}


In [35]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/comprehend/v2/home?region={}#classifier-details/{}/endpoints/{}/details">Comprehend Model Endpoint</a></b>'.format(region, comprehend_training_job_arn, comprehend_endpoint_arn)))

In [36]:
import time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(
        EndpointArn = comprehend_endpoint_arn
    )
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))
    
    if status == "IN_SERVICE" or status == "IN_ERROR":
        break
        
    time.sleep(5)

Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CR

In [37]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.9077000021934509
    },
    {
      "Name": "4",
      "Score": 0.0658000037074089
    },
    {
      "Name": "1",
      "Score": 0.011500000022351742
    }
  ],
  "ResponseMetadata": {
    "RequestId": "a930baeb-8252-4ce4-8491-00b35e31c5da",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "a930baeb-8252-4ce4-8491-00b35e31c5da",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 22 Aug 2020 18:20:48 GMT"
    },
    "RetryAttempts": 0
  }
}


In [38]:
txt = """It's OK."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "3",
      "Score": 0.7184000015258789
    },
    {
      "Name": "4",
      "Score": 0.11620000004768372
    },
    {
      "Name": "2",
      "Score": 0.09910000115633011
    }
  ],
  "ResponseMetadata": {
    "RequestId": "e1995ece-0b5e-40eb-961b-ed5921ad5091",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "e1995ece-0b5e-40eb-961b-ed5921ad5091",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 22 Aug 2020 18:20:48 GMT"
    },
    "RetryAttempts": 0
  }
}


In [39]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.4593999981880188
    },
    {
      "Name": "2",
      "Score": 0.24940000474452972
    },
    {
      "Name": "3",
      "Score": 0.23649999499320984
    }
  ],
  "ResponseMetadata": {
    "RequestId": "39d1b9d3-a267-4f62-a5d6-80cd503cc100",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "39d1b9d3-a267-4f62-a5d6-80cd503cc100",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 22 Aug 2020 18:20:48 GMT"
    },
    "RetryAttempts": 0
  }
}


In [40]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/comprehend/v2/home?region={}#classifier-details/{}/endpoints/{}/details">Comprehend Model Endpoint</a></b>'.format(region, comprehend_training_job_arn, comprehend_endpoint_arn)))

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();